In [1]:
import pandas as pd
import re
import os

# Realtor.com

This is the Realtor.com, Historical Data, All Residential Inventory from [this page](https://www.realtor.com/research/data/) and [this specific link](https://econdata.s3-us-west-2.amazonaws.com/Reports/Core/RDC_Inventory_Core_Metrics_Zip_History.csv)

> It runs from 2016 ⟶ 2020

In [2]:
datadir = '/home/jupyter/data/rei/RealtorDotCom/'
usa = pd.read_csv('gs://datacamp-202518.appspot.com/data/rei/realtor/RDC_Inventory_Core_Metrics_Zip_History.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1,3,14,17,20,23,26,29) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
usa.shape

(607530, 40)

In [4]:
# split into **Five States**:
vt = usa[usa["zip_name"].str.contains(',\svt', na=False)]
ct = usa[usa["zip_name"].str.contains(',\sct', na=False)]
ma = usa[usa["zip_name"].str.contains(',\sma', na=False)]
ri = usa[usa["zip_name"].str.contains(',\sri', na=False)]
nh = usa[usa["zip_name"].str.contains(',\snh', na=False)]

In [5]:
print("Vermont has {} Rows , {} Columns, for all four years".format(vt.shape[0], vt.shape[1]))
print("Connecticut has {} Rows , {} Columns, for all four years".format(ct.shape[0], ct.shape[1]))
print("Massachusetts has {} Rows , {} Columns, for all four years".format(ma.shape[0], ma.shape[1]))
print("Rhode Island has {} Rows , {} Columns, for all four years".format(ri.shape[0], ri.shape[1]))
print("New Hampshire has {} Rows , {} Columns, for all four years".format(nh.shape[0], nh.shape[1]))

Vermont has 3795 Rows , 40 Columns, for all four years
Connecticut has 9850 Rows , 40 Columns, for all four years
Massachusetts has 13048 Rows , 40 Columns, for all four years
Rhode Island has 2380 Rows , 40 Columns, for all four years
New Hampshire has 5211 Rows , 40 Columns, for all four years


In [6]:
vt[["town","state"]] = vt.zip_name.str.split(",",expand=True)
ct[["town","state"]] = ct.zip_name.str.split(",",expand=True)
ma[["town","state"]] = ma.zip_name.str.split(",",expand=True)
ri[["town","state"]] = ri.zip_name.str.split(",",expand=True)
nh[["town","state"]] = nh.zip_name.str.split(",",expand=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [7]:
# dropping colument that we don't need
def dropColumns(x):
    # Fill the ZIP code to be 5 numerals, which means adding the prefix "0"
    x.postal_code = x.postal_code.astype(str)
    x.postal_code = x.postal_code.apply(lambda y: y.zfill(5))
    # Drop the columsn we don't need
    x.drop([ \
         'flag',\
         'zip_name',\
         'active_listing_count',\
         'active_listing_count_mm',\
         'active_listing_count_yy',\
         'median_listing_price_mm',\
         'median_listing_price_yy',\
         'median_days_on_market_mm',\
         'median_days_on_market_yy',\
         'new_listing_count',\
         'new_listing_count_mm',\
         'new_listing_count_yy',\
         'price_increased_count',\
         'price_increased_count_mm',\
         'price_increased_count_yy',\
         'price_reduced_count',\
         'price_reduced_count_mm',\
         'price_reduced_count_yy',\
         'pending_listing_count',\
         'pending_listing_count_mm',\
         'pending_listing_count_yy',\
         'pending_ratio_yy',\
         'median_square_feet',\
         'median_square_feet_mm',\
         'median_square_feet_yy',\
         'average_listing_price',\
         'average_listing_price_mm',\
         'average_listing_price_yy',\
         'total_listing_count',\
         'median_listing_price_per_square_feet_mm',\
         'median_listing_price_per_square_feet_yy',\
         'total_listing_count_mm',\
         'total_listing_count_yy',\
         'pending_ratio',\
         'pending_ratio_mm',\
         'pending_ratio_yy'], axis=1, inplace=True)
    

In [8]:
dropColumns(vt)
dropColumns(ct)
dropColumns(ma)
dropColumns(ri)
dropColumns(nh)

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
vt.dtypes

month_date_yyyymm                        object
postal_code                              object
median_listing_price                    float64
median_days_on_market                   float64
median_listing_price_per_square_feet    float64
town                                     object
state                                    object
dtype: object

In [10]:
vt20 = vt[vt.month_date_yyyymm.astype('str').str.contains('2020', na=False)]
vt19 = vt[vt.month_date_yyyymm.astype('str').str.contains('2019', na=False)]
vt18 = vt[vt.month_date_yyyymm.astype('str').str.contains('2018', na=False)]
vt17 = vt[vt.month_date_yyyymm.astype('str').str.contains('2017', na=False)]
vt16 = vt[vt.month_date_yyyymm.astype('str').str.contains('2016', na=False)]
ct20 = ct[ct.month_date_yyyymm.astype('str').str.contains('2020', na=False)]
ct19 = ct[ct.month_date_yyyymm.astype('str').str.contains('2019', na=False)]
ct18 = ct[ct.month_date_yyyymm.astype('str').str.contains('2018', na=False)]
ct17 = ct[ct.month_date_yyyymm.astype('str').str.contains('2017', na=False)]
ct16 = ct[ct.month_date_yyyymm.astype('str').str.contains('2016', na=False)]
ma20 = ma[ma.month_date_yyyymm.astype('str').str.contains('2020', na=False)]
ma19 = ma[ma.month_date_yyyymm.astype('str').str.contains('2019', na=False)]
ma18 = ma[ma.month_date_yyyymm.astype('str').str.contains('2018', na=False)]
ma17 = ma[ma.month_date_yyyymm.astype('str').str.contains('2017', na=False)]
ma16 = ma[ma.month_date_yyyymm.astype('str').str.contains('2016', na=False)]
nh20 = nh[nh.month_date_yyyymm.astype('str').str.contains('2020', na=False)]
nh19 = nh[nh.month_date_yyyymm.astype('str').str.contains('2019', na=False)]
nh18 = nh[nh.month_date_yyyymm.astype('str').str.contains('2018', na=False)]
nh17 = nh[nh.month_date_yyyymm.astype('str').str.contains('2017', na=False)]
nh16 = nh[nh.month_date_yyyymm.astype('str').str.contains('2016', na=False)]
ri20 = ri[ri.month_date_yyyymm.astype('str').str.contains('2020', na=False)]
ri19 = ri[ri.month_date_yyyymm.astype('str').str.contains('2019', na=False)]
ri18 = ri[ri.month_date_yyyymm.astype('str').str.contains('2018', na=False)]
ri17 = ri[ri.month_date_yyyymm.astype('str').str.contains('2017', na=False)]
ri16 = ri[ri.month_date_yyyymm.astype('str').str.contains('2016', na=False)]

In [11]:
ri20

,month_date_yyyymm,postal_code,median_listing_price,median_days_on_market,median_listing_price_per_square_feet,town,state
269,202003,02806,599050.0,45.5,277.503998,barrington,ri
270,202003,02807,1224550.0,350.5,655.099150,block island,ri
271,202003,02809,497500.0,66.5,305.439926,bristol,ri
272,202003,02813,528050.0,52.5,289.908736,charlestown,ri
273,202003,02814,362450.0,60.5,188.400335,chepachet,ri
...,...,...,...,...,...,...,...
25946,202001,02916,292500.0,68.5,216.949152,rumford,ri
25947,202001,02917,400050.0,60.5,227.199413,smithfield,ri
25948,202001,02919,314950.0,63.0,213.632479,johnston,ri
25949,202001,02920,279000.0,54.0,180.004043,cranston,ri


In [12]:
vt20.to_csv(datadir + 'realtor_vermont_2020.csv')
vt19.to_csv(datadir + 'realtor_vermont_2019.csv')
vt18.to_csv(datadir + 'realtor_vermont_2018.csv')
vt17.to_csv(datadir + 'realtor_vermont_2017.csv')
vt16.to_csv(datadir + 'realtor_vermont_2016.csv')
ct20.to_csv(datadir + 'realtor_connecticut_2020.csv')
ct19.to_csv(datadir + 'realtor_connecticut_2019.csv')
ct18.to_csv(datadir + 'realtor_connecticut_2018.csv')
ct17.to_csv(datadir + 'realtor_connecticut_2017.csv')
ct16.to_csv(datadir + 'realtor_connecticut_2016.csv')
ma20.to_csv(datadir + 'realtor_massachusetts_2020.csv')
ma19.to_csv(datadir + 'realtor_massachusetts_2019.csv')
ma18.to_csv(datadir + 'realtor_massachusetts_2018.csv')
ma17.to_csv(datadir + 'realtor_massachusetts_2017.csv')
ma16.to_csv(datadir + 'realtor_massachusetts_2016.csv')
ri20.to_csv(datadir + 'realtor_rhode-island_2020.csv')
ri19.to_csv(datadir + 'realtor_rhode-island_2019.csv')
ri18.to_csv(datadir + 'realtor_rhode-island_2018.csv')
ri17.to_csv(datadir + 'realtor_rhode-island_2017.csv')
ri16.to_csv(datadir + 'realtor_rhode-island_2016.csv')
nh20.to_csv(datadir + 'realtor_new-hampshire_2020.csv')
nh19.to_csv(datadir + 'realtor_new-hampshire_2019.csv')
nh18.to_csv(datadir + 'realtor_new-hampshire_2018.csv')
nh17.to_csv(datadir + 'realtor_new-hampshire_2017.csv')
nh16.to_csv(datadir + 'realtor_new-hampshire_2016.csv')